# Demo Process Synchronization
Next to the synchronization of activities using controll structures like **while** and **sequence** used within a single process, OpenCLSim allows to synchronize different processes using **start_events**. Start_events can be specified using the expression language as documented to **delay** the execution of a process. Control structures do not delay the execution but explicitly start the execution when possible. The difference is very well visible when you compare the activity log in this demo with the one from the sequence demo.

In [1]:
import datetime, time
import simpy

import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}

## Definition of two basic activities
The two activities are started in parallel, but the execution of Activity2 is delayed until Activity1 has been completed. To easier see how the two activities are interrelated a reporting activity is added.

In [2]:
reporting_activity = model.BasicActivity(
    env=my_env,
    name="Reporting activity",
    registry=registry,
    duration=0,
)
activity = model.BasicActivity(
    env=my_env,
    name="Activity1",
    registry=registry,
    additional_logs=[reporting_activity],
    duration=14,
)
activity2 = model.BasicActivity(
    env=my_env,
    name="Activity2",
    registry=registry,
    additional_logs=[reporting_activity],
    start_event=[{"type": "activity", "name": "Activity1", "state": "done"}],
    duration=30,
)

## Run simulation

In [3]:
model.register_processes([reporting_activity, activity, activity2])
my_env.run()

In [4]:
display(plot.get_log_dataframe(reporting_activity, [activity, activity2, reporting_activity]))

,Activity,Timestamp,ActivityState
0,Reporting activity,1970-01-01 00:00:00,START
1,Activity1,1970-01-01 00:00:00,START
2,Reporting activity,1970-01-01 00:00:00,STOP
3,Activity1,1970-01-01 00:00:14,STOP
4,Activity2,1970-01-01 00:00:00,WAIT_START
5,Activity2,1970-01-01 00:00:14,WAIT_STOP
6,Activity2,1970-01-01 00:00:14,START
7,Activity2,1970-01-01 00:00:44,STOP


Both activities start at the same time. Activity2 gets into a waiting state, which ends, when Activity1 ends. Then Activity2 is executed.